# Site Monitoring and Contour Generation

In [ ]:
import datetime
import json
import os

import cv2
import geojson
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from PIL import Image
import rasterio as rs
from rasterio import warp
from tensorflow import keras
from tqdm import tqdm

from scripts.get_s2_data_ee import get_history, get_history_polygon, get_pixel_vectors, band_descriptions
from scripts.viz_tools import stretch_histogram, create_img_stack, normalize

%load_ext autoreload
%autoreload 2

In [ ]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

In [ ]:
def predict_time_series(patch_histories, site_name, model):
    rgb_stack = []
    preds_stack = []
    dates_list = []
    
    dates = list(patch_histories.keys())
    for date in dates:
        rgb = np.stack((patch_histories[date][site_name]['B4'],
                        patch_histories[date][site_name]['B3'],
                        patch_histories[date][site_name]['B2']), axis=-1)
        
        width, height = rgb.shape[:2]
        pixel_vectors = []
        for i in range(width):
            for j in range(height):
                pixel_vector = []
                band_lengths = [len(patch_histories[date][site_name][band]) for band in band_descriptions]
                if np.array(band_lengths).all() > 0:
                    for band in band_descriptions:
                        pixel_vector.append(patch_histories[date][site_name][band][i][j])
                    pixel_vectors.append(pixel_vector)
        
        pixel_vectors = normalize(pixel_vectors)
        cloudiness = np.sum(rgb <= 0) / np.size(rgb)
        if len(pixel_vectors) > 0 and cloudiness < .05:
            rgb_stack.append(normalize(rgb))
            preds = model.predict(np.expand_dims(pixel_vectors, axis=-1))
            preds_img = np.reshape(preds, (width, height, 2))[:,:,1]
            preds_stack.append(preds_img)
            dates_list.append(date)
            
    return np.array(rgb_stack), np.array(preds_stack), dates_list

def filter_small_points(image):
    # This is experimental. Filter out "hot pixel" predictions
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    mask = cv2.morphologyEx(image, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    out = image * mask
    return out

def green_blue_swap(image):
    # to play nicely with OpenCV's BGR color order
    r,g,b = cv2.split(image)
    image[:,:,0] = b
    image[:,:,1] = g
    image[:,:,2] = r
    return image


def generate_contours(preds_stack, rgb_stack, dates_list, site_coords, threshold=0.5, window_size=3, plot=False):
    """Create contour outlines from """
    # Image upsampling factor. Makes for smoother contours
    scale = 8

    areas = []
    monthly_contours = []
    dates = []
    rgb_contour_imgs = []
    img_size = preds_stack[0].shape
    
    x, y = warp.transform(rs.crs.CRS.from_epsg(4326), rs.crs.CRS.from_epsg(3857), [site_coords[0] - rect_width / 2, 
                                                                               site_coords[0] + rect_width / 2],                                                                      [site_coords[1] - rect_width / 2, 
                                                                               site_coords[1] + rect_width / 2])
    width = abs(x[0] - x[1])
    height = abs(y[0] - y[1])
    img_area_degrees = width * height
    num_pixels = img_size[0] * img_size[1]
    pixel_area = img_area_degrees / num_pixels
    if window_size <= len(preds_stack):
        window_size = len(preds_stack) - 2
    
    for i in range(len(preds_stack) - window_size):
        # Create a median prediction image across predictions within window
        median_pred = (np.median(preds_stack[i:i+window_size], axis=0) * 255).astype(np.uint8)
        # set pixels below threshold to 0
        median_pred[median_pred < threshold * 255] = 0
        
        # Upsample predictions image
        median_pred = np.array(Image.fromarray(median_pred).resize((img_size[0] * scale, img_size[1] * scale)))
        
        # Create a median RGB image
        median_rgb = (np.median(rgb_stack[i:i+window_size], axis=0) * 255).astype(np.uint8)
        median_rgb = np.array(Image.fromarray(median_rgb).resize((img_size[0] * scale, img_size[1] * scale)))
        
        # Construct a binary image with pixels above and below threshold
        ret, thresh = cv2.threshold(median_pred, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # To set threshold manually...
        # thresh_value = threshold * 255 
        # ret, thresh = cv2.threshold(median_pred, thresh_value, 255, 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours with an area below a given threshold
        # I wish I didn't need to do this! It rules out detecting anything smaller than the threshold
        min_pixel_width = 5
        area_threshold = (min_pixel_width * scale) ** 2
        filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > area_threshold]
        monthly_contours.append(filtered_contours)
        
        area = np.sum([cv2.contourArea(contour) for contour in filtered_contours])
        # Convert area to km^2
        area = area * (pixel_area / (1000000 * (scale ** 2)))
        areas.append(area)
        
        # Create images
        three_channel_preds = np.stack((median_pred, median_pred, median_pred), axis=-1)
        preds_contour_img = cv2.drawContours(three_channel_preds, filtered_contours, -1, (255, 0, 0), 2)
        rgb_contour_img = cv2.drawContours(median_rgb, filtered_contours, -1, (255, 0, 0), 2)
        rgb_contour_imgs.append(rgb_contour_img)

        date = dates_list[i + window_size]
        dates.append(date)

        if plot:
            plt.figure(figsize=(10,5), dpi=150)
            plt.subplot(1,2,1)
            plt.title(date + ' rgb')
            plt.imshow(rgb_contour_img, vmax=255, vmin=0)
            plt.axis('off')
            plt.subplot(1,2,2)
            plt.imshow(preds_contour_img, vmax=255, vmin=0)
            plt.title(date + ' pred')
            plt.axis('off')
            plt.show()
    
    return monthly_contours, areas, dates, rgb_contour_imgs, scale

def contours_to_geojson(monthly_contours, name, areas, date_list, site_coords, img_size, scale):
    
    # Define patch coordinate bounds to set pixel scale
    west  = site_coords[0] - rect_width / 2
    east  = site_coords[0] + rect_width / 2
    north = site_coords[1] + rect_width / 2
    south = site_coords[1] - rect_width / 2
    transform = rs.transform.from_bounds(west, south, east, north, img_size[0] * scale, img_size[1] * scale)

    polygons = []
    for contours, area, date in zip(monthly_contours, areas, date_list):
        polygon_coords = []
        for contour in contours:
            # Convert from pixels to coords
            contour_coords = []
            for point in contour[:,0]:
                lon, lat = rs.transform.xy(transform, point[1], point[0])
                contour_coords.append([lon, lat])
            # Close the loop
            contour_coords.append(contour_coords[0])

            # Add individual contour to list of contours for the month
            polygon_coords.append(contour_coords)

        date = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y/%m/%d %H:%M")
        polygon = geojson.MultiPolygon(coordinates = [polygon_coords])
        contour_feature = geojson.Feature(geometry=polygon, properties={'date': date,
                                                                        'name': name,
                                                                        'area': area
                                                                       })
        polygons.append(contour_feature)
        
    return polygons

def plot_site_area(areas, name, site_coords, img_size, dates, scale, path):
    plt.figure(figsize=(8,4), facecolor=(1,1,1), dpi=150)
    months = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]
    plt.plot(months, areas)
    
    plt.xticks(ha='right', rotation=45)
    plt.title(name + ' Landfill Area over Time')
    plt.ylabel('Area (km^2)')
    plt.xlabel('Date')
    plt.savefig(path, bbox_inches='tight')
    plt.close('all')

def monitor_sites(patch_history, model, names, coords, region_name, threshold=0.5, window_size=3):
    region_dir = os.path.join('../data/model_outputs/site_contours/monthly_contours', f'{region_name}_thresh_{threshold}_window_{window_size}')
    if not os.path.exists(region_dir):
        os.mkdir(region_dir)
            
    contour_collection = []
    
    for name, site_coords in tqdm(zip(names, coords)):

        data_dir = os.path.join(region_dir, name)
        if not os.path.exists(data_dir):
            os.mkdir(data_dir)
        
        rgb_stack, preds_stack, dates_list = predict_time_series(patch_history, name, model)
        img_size = preds_stack[0].shape
        
        # Compute contours
        monthly_contours, areas, dates, rgb_contour_imgs, scale = generate_contours(preds_stack, rgb_stack, dates_list, site_coords, threshold=threshold, window_size=window_size, plot=False)
        
        # Write contour overlay images
        for img, date in zip(rgb_contour_imgs, dates):
            cv2.imwrite(os.path.join(data_dir, f'{name}_contours_window_size_{window_size}_thresh_{threshold}_{date.replace("/", "-")[:10]}.png'), cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        # Plot site area
        plot_site_area(areas, name, site_coords, img_size, dates, scale, os.path.join(data_dir, f'{name}_site_area_window_size_{window_size}_thresh_{threshold}.png'))
        
        # Convert contours to GeoJSON
        median_contour = contours_to_geojson(monthly_contours, name, areas, dates, site_coords, img_size, scale)
        contour_collection.append(median_contour)
        json_contours = json.dumps(geojson.FeatureCollection(median_contour))
        
        # Write site-specific contour
        with open(os.path.join(data_dir, f'{name}_contours_window_size_{window_size}_thresh_{threshold}.geojson'), 'w') as f:
            f.write(json_contours)
    
    # Write master GeoJSON for all contours in patch_history
    json_contour_collection = json.dumps(geojson.FeatureCollection(contour_collection))
    with open(os.path.join('../data/model_outputs/site_contours', f'{region_name}_contours_window_size_{window_size}_thresh_{threshold}.geojson'), 'w') as f:
        f.write(json_contour_collection)

## Download Data

In [ ]:
# Enter rect width in degrees (0.035 max recommended) and site coordinates
rect_width = 0.075

### Download Single Site

In [ ]:
num_months = 12
start_date = '2019-05-01'
site_coords = [115.2211451864497, -8.723290447230358]
name = 'TPA Suwung'
patch_history = get_history([site_coords], 
                            [name],
                            rect_width,
                            num_months = num_months,
                            start_date = start_date,
                            cloud_mask = True)

### Download all Bali TPA Data

In [ ]:
import json
with open('../data/sampling_locations/tpa_points.geojson', 'r') as f:
    tpa_sites = json.load(f)
coords = [site['geometry']['coordinates'] for site in tpa_sites['features']]
names = [site['properties']['Name'] for site in tpa_sites['features']]

num_months = 60
start_date = '2016-01-01'
patch_history = get_history(coords, 
                          names, 
                          rect_width,
                          num_months=num_months,
                          start_date=start_date,
                          cloud_mask=True)

### Download all Java TPA Data

In [ ]:
import pandas as pd
java = pd.read_csv('../data/sampling_locations/v12_java_validated_positives.csv')
java_names = list(java['name'])
java_coords = [[lon, lat] for lon, lat in zip(java['lon'], java['lat'])]
num_months = 24
start_date = '2019-01-01'
patch_history = get_history(java_coords, 
                          java_names, 
                          rect_width,
                          num_months=num_months,
                          start_date=start_date,
                          cloud_mask=True)

### Load data that has already been downloaded

In [ ]:
import pickle
import json
with open('../data/sampling_locations/tpa_points.geojson', 'r') as f:
    tpa_sites = json.load(f)
coords = [site['geometry']['coordinates'] for site in tpa_sites['features']]
names = [site['properties']['Name'] for site in tpa_sites['features']]

num_months = 60
start_date = '2016-01-01'
with open('../data/training_data/patch_histories/bali_tpa_raw_60_months_2016-01-01_84px_patch_history.pkl', 'rb') as f:
    patch_history = pickle.load(f)

In [ ]:
import pandas as pd
java = pd.read_csv('../data/sampling_locations/v12_java_validated_positives.csv')
names = list(java['name'])
coords = [[lon, lat] for lon, lat in zip(java['lon'], java['lat'])]
num_months = 24
start_date = '2019-01-01'

with open('../data/training_data/patch_histories/java_v12_detections_raw_24_months_2019-01-01_84px_patch_history.pkl', 'rb') as f:
    patch_history = pickle.load(f)

# Generate Contours

In [ ]:
# Select model to load for predictions
model = keras.models.load_model('../models/65_mo_tpa_bootstrap_toa-12-20-2020.h5')

In [ ]:
monitor_sites(patch_history, model, names, coords, 'java_v12', threshold=0.6, window_size=8)

## WIP section to fix contour export

In [ ]:
import json
with open('../data/model_outputs/site_contours/java_v12_contours_window_size_3_thresh_0.6.geojson', 'r') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
feature[0]

In [ ]:
month

In [ ]:
data['features'][1]

In [ ]:
import geojson
feature_list = []
for feature in data['features']:
    for month in feature:
        feature_list.append(month)
fc = geojson.FeatureCollection(feature_list)

In [ ]:
with open('../data/model_outputs/site_contours/java_v12_contours_window_size_3_thresh_0.6_fix.geojson', 'w') as f:
    f.write(json.dumps(fc))